<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    استارت‌آپ
</font>
</h1>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در این سوال می‌خواهیم مدلی طراحی کنیم تا بتواند پیش‌بینی گند یک استارت‌آپ می‌تواند موفق شود یا خیر.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در فایل اولیه‌ی این سوال دو فایل با نام‌های <code>train.csv</code> و <code>test.csv</code> قرار دارد که به ترتیب مجموعه‌داده‌ی آموزش و آزمون هستند.
    <br>
    مجموعه‌داده‌ی آموزش دارای ۵۷۶۱۵ سطر و ۱۲ ستون (ویژگی) است که توضیحات مربوط به این ستون‌ها در جدول زیر آمده است.
</font>
</p>

<center>
<div dir=rtl align=center style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

| <b>نام ویژگی</b> | <b>توضیح ویژگی</b> |
| :---: | :---: |
| <code>name</code> | نام شرکت |
| <code>category_list</code> | زمینه‌ی کاری شرکت |
| <code>funding_total_usd</code> | مجموع بودجه‌ی شرکت (به دلار) |
| <code>status</code> | وضعیت شرکت (متغیر هدف مسئله که باید در ادامه کمی تغییرات در آن ایجاد کنید) |
| <code>country_code</code> | کد کشور |
| <code>state_code</code> | کد ایالت |
| <code>region</code> | منطقه |
| <code>city</code> | شهر |
| <code>funding_rounds</code> | تعداد دفعات تامین مالی شرکت |
| <code>founded_at</code> | تاریخ تاسیس |
| <code>first_funding_at</code> | تاریخ اولین تامین مالی شرکت |
| <code>last_funding_at</code> | تاریخ آخرین تامین مالی شرکت |

</font>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    مجموعه‌داده‌ی آزمون نیز دارای ۸۷۵۲ سطر است و ستون‌های آن مشابه مجموعه‌داده‌ی آموزش است، با این تفاوت که ستون <code>status</code> را ندارد.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است تا کتابخانه‌های مورد نیاز خود را فراخوانی کنید. سپس با توجه به توضیحات بالا مجموعه‌داده‌های آموزش و آزمون را به نحو مناسبی بخوانید و پیش‌پردازش‌های لازم را روی آن‌ها انجام دهید.
    <br>
    اگر به داده‌ها دقت کنید مقادیر ستون <code>status</code> برابر با <code>operating</code> ،<code>closed</code> ،<code>acquired</code> و <code>ipo</code> است. 
    شرکتی را موفق می‌گوییم اگر در یکی از دو وضعیت <code>acquired</code> و <code>ipo</code> باشد. وضعیت <code>closed</code> به معنی این است که استارت‌آپ شکست خورده و شرکت بسته شده است و وضعیت <code>operating</code> نیز به این معنی است که شرکت به موفقیت نرسیده ولی هنوز ورشکست نشده است.
     بنابراین مدل شما در نهایت باید به عنوان پیش‌بینی یکی از سه عدد ۰ (شکست و بسته شده)، ۱ (عدم موفقیت ولی بشته نشده) و ۲ (موفقیت) را خروجی دهد.
</font>
</p>

In [68]:

# Classification imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier

# Optional: visualization settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)


In [69]:
train = pd.read_csv("./train.csv",parse_dates=["first_funding_at","last_funding_at","founded_at"])
x_test = pd.read_csv("./test.csv",parse_dates=["first_funding_at","last_funding_at","founded_at"])

In [70]:
train.drop("name",axis=1,inplace=True)
x_test.drop("name",axis=1,inplace=True)

In [71]:
train

,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24
...,...,...,...,...,...,...,...,...,...,...,...
57611,Enterprise Software,1587301,operating,CHN,22,Beijing,Beijing,1,NaN,2012-04-01,2012-04-01
57612,Advertising|Mobile|Web Development|Wireless,114304,operating,HRV,15,Split,Split,4,2012-05-13,2011-11-01,2014-03-01
57613,NaN,-,operating,NaN,NaN,NaN,NaN,1,2011-01-01,2014-08-01,2014-08-01
57614,Consumer Electronics|Internet of Things|Teleco...,18192,operating,USA,CA,SF Bay Area,San Francisco,1,2014-01-01,2015-01-01,2015-01-01


In [72]:
train.isnull().sum() / train.shape[0] *100 

category_list         5.463760
funding_total_usd     0.000000
status                0.000000
country_code         12.076507
state_code           14.834421
region               13.937101
city                 13.933630
funding_rounds        0.000000
founded_at           26.418009
first_funding_at      0.041655
last_funding_at       0.000000
dtype: float64

In [73]:
train["first_funding_at"] = pd.to_datetime(train["first_funding_at"], errors="coerce")
train["founded_at"] = pd.to_datetime(train["founded_at"], errors="coerce")
train.replace('-', np.nan, inplace=True)
x_test.replace('-', np.nan, inplace=True)
train["funding_total_usd"] = train["funding_total_usd"].astype(np.float64)
x_test["funding_total_usd"] = x_test["funding_total_usd"].astype(np.float64)
train

,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,Media,10000000.0,operating,IND,16,Mumbai,Mumbai,1,NaT,2015-01-05,2015-01-05
1,Application Platforms|Real Time|Social Network...,700000.0,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,Apps|Games|Mobile,3406878.0,operating,NaN,NaN,NaN,NaN,1,NaT,2014-01-30,2014-01-30
3,Curated Web,2000000.0,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,Software,NaN,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24
...,...,...,...,...,...,...,...,...,...,...,...
57611,Enterprise Software,1587301.0,operating,CHN,22,Beijing,Beijing,1,NaT,2012-04-01,2012-04-01
57612,Advertising|Mobile|Web Development|Wireless,114304.0,operating,HRV,15,Split,Split,4,2012-05-13,2011-11-01,2014-03-01
57613,NaN,NaN,operating,NaN,NaN,NaN,NaN,1,2011-01-01,2014-08-01,2014-08-01
57614,Consumer Electronics|Internet of Things|Teleco...,18192.0,operating,USA,CA,SF Bay Area,San Francisco,1,2014-01-01,2015-01-01,2015-01-01


In [74]:
mask = train["founded_at"].notna()
train = train[mask]
mask = x_test["founded_at"].notna()
x_test = x_test[mask]

In [75]:
from sklearn.preprocessing import OrdinalEncoder

str_cols = ["country_code", "state_code", "region", "city"]
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
train[str_cols] = encoder.fit_transform(train[str_cols])

x_test[str_cols] = encoder.transform(x_test[str_cols])
train

C:\Users\SH.Rayan\AppData\Local\Temp\ipykernel_19068\3825761827.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[str_cols] = encoder.fit_transform(train[str_cols])


,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
1,Application Platforms|Real Time|Social Network...,700000.0,operating,119.0,120.0,219.0,887.0,2,2014-09-04,2014-03-01,2014-10-14
3,Curated Web,2000000.0,operating,21.0,14.0,77.0,251.0,1,2007-01-01,2008-03-19,2008-03-19
4,Software,NaN,operating,119.0,161.0,817.0,613.0,1,2010-01-01,2014-07-24,2014-07-24
6,Biotechnology,762851.0,operating,18.0,102.0,914.0,3593.0,2,1997-01-01,2009-09-11,2009-12-21
7,Analytics,33600000.0,operating,119.0,110.0,752.0,2298.0,4,2011-01-01,2013-01-03,2015-11-09
...,...,...,...,...,...,...,...,...,...,...,...
57607,Biotechnology,15419877.0,closed,82.0,5.0,539.0,847.0,4,1994-01-01,2008-01-01,2013-02-15
57610,Biotechnology,2686600.0,operating,5.0,22.0,922.0,1257.0,1,2007-01-01,2013-01-29,2013-01-29
57612,Advertising|Mobile|Web Development|Wireless,114304.0,operating,49.0,6.0,814.0,3267.0,4,2012-05-13,2011-11-01,2014-03-01
57613,NaN,NaN,operating,NaN,NaN,NaN,NaN,1,2011-01-01,2014-08-01,2014-08-01


In [76]:
train.dtypes

category_list                object
funding_total_usd           float64
status                       object
country_code                float64
state_code                  float64
region                      float64
city                        float64
funding_rounds                int64
founded_at           datetime64[ns]
first_funding_at     datetime64[ns]
last_funding_at      datetime64[ns]
dtype: object

In [77]:
def extract_date_features(df):
    df["time_to_first_funding"] = (df["first_funding_at"] - df["founded_at"]).dt.days
    df["funding_lifespan"] = (df["last_funding_at"] - df["first_funding_at"]).dt.days
    df["age_at_last_funding"] = (df["last_funding_at"] - df["founded_at"]).dt.days
    df["founded_at_year"] = df["founded_at"].dt.year
    df["founded_at_month"] = df["founded_at"].dt.month
    df["founded_at_quarter"] = df["founded_at"].dt.quarter
    df["founded_at_day_of_week"] = df["founded_at"].dt.day_of_week
    df["first_funding_at_year"] = df["first_funding_at"].dt.year
    df["first_funding_at_month"] = df["first_funding_at"].dt.month
    df["first_funding_at_quarter"] = df["first_funding_at"].dt.quarter
    df["first_funding_at_day_of_week"] = df["first_funding_at"].dt.day_of_week
    df["last_funding_at_year"] = df["last_funding_at"].dt.year
    df["last_funding_at_month"] = df["last_funding_at"].dt.month
    df["last_funding_at_quarter"] = df["last_funding_at"].dt.quarter
    df["last_funding_at_day_of_week"] = df["last_funding_at"].dt.day_of_week

extract_date_features(train)

extract_date_features(x_test)

C:\Users\SH.Rayan\AppData\Local\Temp\ipykernel_19068\700633556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time_to_first_funding"] = (df["first_funding_at"] - df["founded_at"]).dt.days
C:\Users\SH.Rayan\AppData\Local\Temp\ipykernel_19068\700633556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["funding_lifespan"] = (df["last_funding_at"] - df["first_funding_at"]).dt.days
C:\Users\SH.Rayan\AppData\Local\Temp\ipykernel_19068\700633556.py:4: SettingWithCopyWarning: 
A value is trying to be 

In [78]:
train.drop(columns=["founded_at", "first_funding_at", "last_funding_at"], inplace=True)

x_test.drop(columns=["founded_at", "first_funding_at", "last_funding_at"], inplace=True)


C:\Users\SH.Rayan\AppData\Local\Temp\ipykernel_19068\224011857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(columns=["founded_at", "first_funding_at", "last_funding_at"], inplace=True)


In [79]:
from sklearn.impute import SimpleImputer

continuous_features = [
    'funding_total_usd',
    'time_to_first_funding',
    'age_at_last_funding'
]

categorical_features = [
    'country_code',
    'state_code',
    'region',
    'city',
    'founded_at_year',
    'founded_at_month',
    'founded_at_quarter',
    'founded_at_day_of_week'
]

continuous_imputer = SimpleImputer(strategy="median")
categorical_imputer = SimpleImputer(strategy="most_frequent")

preprocessor = ColumnTransformer(
    transformers=[
        ('continuous_imputer', continuous_imputer, continuous_features),
        ('categorical_imputer', categorical_imputer, categorical_features)
    ],
    remainder='passthrough'
)


train = pd.DataFrame(preprocessor.fit_transform(train), columns=preprocessor.get_feature_names_out(), index=train.index)
x_test = pd.DataFrame(preprocessor.fit_transform(x_test), columns=preprocessor.get_feature_names_out(), index=x_test.index)



In [80]:
print(train.columns.tolist())


['continuous_imputer__funding_total_usd', 'continuous_imputer__time_to_first_funding', 'continuous_imputer__age_at_last_funding', 'categorical_imputer__country_code', 'categorical_imputer__state_code', 'categorical_imputer__region', 'categorical_imputer__city', 'categorical_imputer__founded_at_year', 'categorical_imputer__founded_at_month', 'categorical_imputer__founded_at_quarter', 'categorical_imputer__founded_at_day_of_week', 'remainder__category_list', 'remainder__status', 'remainder__funding_rounds', 'remainder__funding_lifespan', 'remainder__first_funding_at_year', 'remainder__first_funding_at_month', 'remainder__first_funding_at_quarter', 'remainder__first_funding_at_day_of_week', 'remainder__last_funding_at_year', 'remainder__last_funding_at_month', 'remainder__last_funding_at_quarter', 'remainder__last_funding_at_day_of_week']


In [81]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# split category strings into lists
train["remainder__category_list"] = train["remainder__category_list"].map(
    lambda x: x.split("|") if isinstance(x, str) else []
)
x_test["remainder__category_list"] = x_test["remainder__category_list"].map(
    lambda x: x.split("|") if isinstance(x, str) else []
)

# fit on train, transform both
train_encoded = pd.DataFrame(
    mlb.fit_transform(train["remainder__category_list"]),
    columns=[f'category_list_{c}' for c in mlb.classes_]
)

x_test_encoded = pd.DataFrame(
    mlb.transform(x_test["remainder__category_list"]),
    columns=[f'category_list_{c}' for c in mlb.classes_]
)

# merge back
train = pd.concat([train.reset_index(drop=True), train_encoded], axis=1)
x_test = pd.concat([x_test.reset_index(drop=True), x_test_encoded], axis=1)


c:\Users\SH.Rayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['Building Owners', 'Hi Tech', 'Hip Hop'] will be ignored
  warnings.warn(


In [ ]:
train.drop(columns="remainder__category_list",inplace=True,axis=1)
x_test.drop(columns="remainder__category_list",inplace=True,axis=1)

KeyError: "['reminder__category_list'] not found in axis"

In [ ]:
X = train.drop(columns=["status"])
Y = train["status"].map({"closed": 0, "operating": 1, "acquired": 2, "ipo": 2})
x_train,x_val,y_train,y_val = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1️⃣ Align val/test columns to match training columns
x_val  = x_val.reindex(columns=x_train.columns, fill_value=0)
x_test = x_test.reindex(columns=x_train.columns, fill_value=0)

# 2️⃣ Fit only on train, transform all
std = StandardScaler()

x_train = pd.DataFrame(std.fit_transform(x_train), columns=x_train.columns, index=x_train.index)
x_val   = pd.DataFrame(std.transform(x_val), columns=x_train.columns, index=x_val.index)
x_test  = pd.DataFrame(std.transform(x_test), columns=x_train.columns, index=x_test.index)


In [ ]:
y_train.value_counts()

status
1    27484
2     3608
0     2820
Name: count, dtype: int64

In [ ]:
x_train.isnull().sum()

funding_total_usd           6238
country_code                2968
state_code                  3837
region                      3461
city                        3460
                            ... 
category_list_iOS              0
category_list_iPad             0
category_list_iPhone           0
category_list_iPod Touch       0
category_list_mHealth          0
Length: 869, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    حال که داده را پاکسازی کرده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
</font>
</p>

In [ ]:
# Model design
# TODO

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>f1_score</code> (با روش میانگین‌گیری <code>macro</code>) نام دارد.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
    برای دریافت نمره از این سوال لازم است تا دقت مدل شما از آستانه‌ی ۰.۴ بیشتر باشد.
    در صورتی که دقت مدل شما از ۰.۴ کمتر باشد نمره شما 
    <b>صفر</b>
    خواهد شد و در غیر این صورت با فرمول زیر محاسبه می‌شود:
</font>
</p>


In [ ]:
# evaluate your model
from sklearn.metrics import f1_score

# To-Do

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی بر روی داده‌ی تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پیش‌بینی مدل خود بر روی داده‌های آزمون را در یک دیتافریم (<code>dataframe</code>) به فرمت زیر ذخیره کنید.
</font>
</p>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    توجه داشته باشید که نام دیتافریم باید <code>submission</code> باشد؛ در غیر این‌صورت، سامانه‌ی داوری قادر به ارزیابی خروجی شما نخواهد بود.
    این دیتافریم صرفا شامل یک ستون به نام <code>status</code> است و ۸۷۵۲ سطر دارد.
    <br>
    به ازای هر سطر موجود در مجموعه‌داده‌ی آزمون، باید یک مقدار پیش‌بینی‌شده داشته باشید که مقدار <code>status</code> پیش‌بینی مدل شما است.
    به‌عنوان مثال جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته این مقادیر به‌صورت فرضی هستند و در جواب شما، ممکن است متفاوت باشند.
</font>
</p>

<center>
<div align=center 
style="direction: ltr;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||<code>status</code>|
|:----:|:-----:|
|0|1|
|1|2|
|2|1|
|3|1|
|4|0|

</font>
</div>
</center>

In [ ]:
# To-Do
submission =

SyntaxError: invalid syntax (995641694.py, line 2)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [ ]:
import zipfile

if not os.path.exists(os.path.join(os.getcwd(), 'startup.ipynb')):
    %notebook -e startup.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)

file_names = ['startup.ipynb', 'submission.csv']
compress(file_names)